In [2]:
import numpy as np
np.random.seed(10)
import tensorflow as tf
tf.random.set_seed(10)
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import matplotlib.pyplot as plt

2022-10-15 13:18:34.560890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 13:18:36.561070: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-15 13:18:43.715693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-15 13:18:43.715784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [1]:
import glob

In [3]:
# In physical space use 'raw_train.reshape(-1,121,281)' to visualize original
num_snapshots = 12564
directory = '/lcrc/project/AIEADA-2/era5_data/full_data/snapshots/'

#var_list = [raw_train_z500,
#            raw_train_u250,raw_train_v250,raw_train_t250,
#            raw_train_u850,raw_train_v850,raw_train_t850,
#            raw_train_blh,raw_train_tcwv]
input_window = 14
output_window = 7

In [45]:
def load_data(file_path):
    print(file_path)
    input_window = 14
    example = tf.py_function(np.load(file_path))
    input_data = example[:input_window,:]
    output_data = example[input_window:,:]
    return input_data, output_data

In [46]:
def build_dataset(batch_size):
    dataset_dir = directory + 'split_examples/train_data/'
    filelist = glob.glob(dataset_dir + '*.npy')
    filelist = tf.data.Dataset.from_tensor_slices(filelist)
    shuffle_buffer = 6000
    filelist = filelist.shuffle(shuffle_buffer,reshuffle_each_iteration=True)
    ds = filelist.map(load_data,
                     num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # batch the data
    ds = ds.batch(batch_size)

In [47]:
batch_size = 512
build_dataset(batch_size)

Tensor("args_0:0", shape=(), dtype=string)


StagingError: in user code:

    File "/tmp/ipykernel_2939936/2282494107.py", line 4, in load_data  *
        example = tf.py_function(np.load(str(file_path)))
    File "/home/blusch/.conda/envs/swing_env/lib/python3.10/site-packages/numpy/lib/npyio.py", line 390, in load  **
        fid = stack.enter_context(open(os_fspath(file), "rb"))

    FileNotFoundError: [Errno 2] No such file or directory: 'Tensor("args_0:0", shape=(), dtype=string)'


In [ ]:
embed_dim = train_inputs[0].shape[-1]
output_dim = train_outputs[0].shape[-1]
encode_dim = 180

In [ ]:
num_ae_encoder_layers = 3
num_ae_decoder_layers = 3
num_lstm_cells_encoder = 2
num_lstm_cells_decoder = 2

In [ ]:
ff_dim = 100  # Hidden layer size in feed forward network inside transformer
dropout_rate = 0.0

inputs = layers.Input(shape=(input_window,embed_dim))
ae_encoding_layers = []
for _ in range(num_ae_encoder_layers):
    ae_encoding_layers.append(layers.TimeDistributed(layers.Dense(encode_dim,activation='elu')))

lstm_encoder_cells = []
for _ in range(num_lstm_cells_encoder):
    lstm_encoder_cells.append(layers.Bidirectional(layers.LSTM(ff_dim,activation='elu',return_sequences=True,)))

lstm_encoder_final = layers.Bidirectional(layers.LSTM(ff_dim,activation='elu'))
lstm_repeater_layer = layers.RepeatVector(output_window)
    
lstm_decoder_cells = []
for _ in range(num_lstm_cells_decoder):
    lstm_decoder_cells.append(layers.LSTM(encode_dim,activation='elu',return_sequences=True,))

    
ae_decoding_layers = []
for _ in range(num_ae_decoder_layers):
    ae_decoding_layers.append(layers.TimeDistributed(layers.Dense(embed_dim,activation='elu')))


# Encode from physical space
print('Input shape:',inputs.get_shape().as_list())

x = inputs
for i in range(num_ae_encoder_layers):
    x = ae_encoding_layers[i](x)
encoded = x

print('AE Encoded shape:',encoded.get_shape().as_list())

x = lstm_encoder_cells[0](encoded)    
for i in range(1,num_lstm_cells_encoder):
    x = lstm_encoder_cells[i](x)
    
x = lstm_encoder_final(x)

print('LSTM Encoded shape:',x.get_shape().as_list())

x = lstm_repeater_layer(x)

for i in range(num_lstm_cells_decoder):
    x = lstm_decoder_cells[i](x)

print('LSTM Decoded shape:',x.get_shape().as_list())
    
for i in range(num_ae_decoder_layers):
    x = ae_decoding_layers[i](x)
    
outputs = x

print('AE+LSTM Output shape:',outputs.get_shape().as_list())
    
decoded = encoded
for i in range(num_ae_decoder_layers):
    decoded = ae_decoding_layers[i](decoded)
    
print('AE Output shape:',decoded.get_shape().as_list())

model = tf.keras.Model(inputs=inputs, outputs=[outputs,decoded])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss',patience=20)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='mean_squared_error',loss_weights=[1,1])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_inputs,[train_outputs,train_inputs],epochs=250,batch_size=2,callbacks=[reduce_lr,early_stop],validation_split=0.2)